# Demo: Convert single basestation *.nc file

This notebook converts sg015 data from the Labrador Sea in September 2024 into OG1 format.

- OG1 format is a newly agreed format (since June 2024) for glider data sets from various platforms (e.g., Seaglider, Slocum, Seaexplorer).  It lives on github here: (https://github.com/OceanGlidersCommunity/OG-format-user-manual).
- OG1 manual: https://oceangliderscommunity.github.io/OG-format-user-manual/OG_Format.html

Process:

1. For one basestation dataset, split the dataset by dimension (`split_ds`)
3. Transform into OG1 format: dataset with dims `sg_data_point`
    - Change the dimension to `N_MEASUREMENTS`
    - Rename variables according to `vocabularies.standard_names` 
    - Assign variable attributes according to `vocabularies.vocab_attrs`.  (Note: This *could* go wrong since it makes assumptions about the input variables. May need additional handling.)
4. Add missing mandatory variables: 
    - From `split_ds[(gps_info,)]`, add the `LATITUDE_GPS`, `LONGITUDE_GPS` and `TIME_GPS` (Note: presently `TIME_GPS` is stripped before saving, but `TIME` values contain `TIME_GPS`)
    - Create `PROFILE_NUMBER` and `PHASE`
    - Calculate `DEPTH_Z` which is positive up
5. Update attributes for the file. 
    - Combines `creator` and `contributor` from original attributes into `contributor`
    - Adds `contributing_institutions` based on `institution`
    - Reformats time in `time_coverage_*` and `start_time`--> `start_date`
    - Adds `date_modified`
    - Renames `comments`-->`history`, `site`-->`summary`
    - Adds `title`, `platform`, `platform_vocabulary`, `featureType`, `Conventions`, `rtqc_method*` according to OceanGliders format
    - Retains `naming_authority`, `institution`, `project`, `geospatial_*` as OG attributes
    - Retains extra attributes: `license`, `keywords`, `keywords_vocabulary`, `file_version`, `acknowledgement`, `date_created`, `disclaimer`

Future behaviour to be added:

6. Retain the variables starting with `sg_cal` and check whether they vary over the mission (shouldn't)
6. Add sensors, using information in the `split_ds` with no dimensions
    - Need (from sg_cal_constants: `sg_cal` plus `volmax`, `vbd_cnts_per_cc`, `therm_expan`, `t_*`, `mass`, `hd_*`, `ctcor`, `cpcor`, `c_*`, `abs_compress`, `a`, `Tcor`, `Soc`, `Pcor`, `Foffset`)
    - Maybe also `reviewed`, `magnetic_variation` (which will change with position), `log_D_FLARE`, `flight_avg_speed_north` and `flight_avg_speed_east` also with `_gsm`, `depth_avg_curr_north` and `depth_avg_curr_east` also with `_gsm`, 
    `wlbb2f` - means sensor
    `sg_cal_mission_title`
    `sg_cal_id_str`
    `calibcomm_oxygen`
    `calibcomm`
    `sbe41` means ??
    `hdm_qc`
    `glider`

In [ ]:
# This cell is for development purposes
import sys
# Change to your path
sys.path.append('/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1')

In [ ]:
import matplotlib.pyplot as plt
import yaml
from seagliderOG1 import plotters
from seagliderOG1 import readers
from seagliderOG1 import convertOG1
from seagliderOG1 import vocabularies
import xarray as xr
import os
import warnings
warnings.simplefilter("ignore", category=Warning)

In [ ]:
# Add these to existing attributes - update to your details
contrib_to_append = {
    'contributor_name': 'Eleanor Frajka-Williams',
    'contributor_email': 'eleanorfrajka@gmail.com',
    'contributor_role': 'Data scientist',
    'contributor_role_vocabulary': 'http://vocab.nerc.ac.uk/search_nvs/W08',
    'contributing_institutions': 'University of Hamburg - Institute of Oceanography',
    'contributing_institutions_vocabulary': 'https://edmo.seadatanet.org/report/1156',
    'contributing_institutions_role': 'Data scientist',
    'contributing_institutions_role_vocabulary': 'http://vocab.nerc.ac.uk/search_nvs/W08',
}

## Load Seaglider data in basestation format

Test case build on a file which was written in 2013 by basestation v2.8 into nodc format template v0.9.

In [ ]:
# Specify the server where data are located
if 1:
    input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/"
    input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100729/"
else:
    input_loc = '/Users/eddifying/Dropbox/data/sg015-ncei-download/'

# Load datasets from the server, optionally specifying the range of profiles to load
start_dive = 1
end_dive = 10

# --- List the data
list_datasets = readers.read_basestation(input_loc, 1, 10)

# Pick one basestation xarray dataset to work with
ds1 = list_datasets[0]

# --- Convert the data
ds_single_OG1 = convertOG1.convert_to_OG1(list_datasets, contrib_to_append)

# --- Output file
# Check a location for the output file
output_file = os.path.join('../data', 'demo_single_test.nc')
# If it's already there, remove it first
if os.path.exists(output_file):
    os.remove(output_file)

# Write the file
# This writer catches errors in data types (DType errors) when using xr.to_netcdf()
# The solution is to convert them to strings, which may be undesired behaviour
convertOG1.save_dataset(ds_single_OG1, output_file)

### Same process, but breaking out sub-functions

This is the same process as above (contained in `convertOG1.convert_to_OG1`), but breaking out to access the sub-functions individually.  This way you can inspect the process as it goes along, and also inspect some of the data which did not make it into the final dataset:

- `sg_cal` - details from `sg_calib_constants.m`, 
- `dc_log` - log events, and 
- `dc_other` - random other variables that were in the basestation file).

In [ ]:
# Convert the dataset and output also variables not included
ds_new, attr_warnings, sg_cal, dc_other, dc_log = convertOG1.process_dataset(ds1)

# Create the list of attributes in order
ordered_attributes = convertOG1.update_dataset_attributes(ds1, contrib_to_append)

for key, value in ordered_attributes.items():
    ds_new.attrs[key] = value

# Construct the platform serial number
PLATFORM_SERIAL_NUMBER = 'sg' + ds_new.attrs['id'][1:4]
ds_new['PLATFORM_SERIAL_NUMBER'] = PLATFORM_SERIAL_NUMBER
ds_new['PLATFORM_SERIAL_NUMBER'].attrs['long_name'] = "glider serial number"

# Construct the unique identifier attribute
id = f"{PLATFORM_SERIAL_NUMBER}_{ds_new.start_date}_delayed"
ds_new.attrs['id'] = id


### Save the data 

As above, but builds the filename out of parameters in the file.


In [ ]:
plotters.show_attributes(ds_new)
## Save the dataset to a NetCDF file
# Check if the file exists and delete it if it does

output_file = os.path.join('../data', 'demo_single_test2.nc')
if os.path.exists(output_file):
    os.remove(output_file)

# Save the dataset to a NetCDF file
convertOG1.save_dataset(ds_new, output_file)
